This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [1]:
# import os

# os.environ["OPENAI_API_KEY"] = "YOUR API KEY"
# os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

# from langchain.llms import OpenAI

import os
from real_estate_db import RealEstateDBManager
from embedding_utils import get_embedder
from rag_pipeline import RAGEngine

In [8]:
DB_PATH = os.getenv("LANCEDB_PATH", "../../../../data/GenAI/05_project/lancedb_store")
JSON_PATH = os.getenv("LISTINGS_JSON", "listings.json")  # Listings file

In [9]:
manager = RealEstateDBManager(DB_PATH)

In [10]:
# create_table(force=True) will drop old table.
manager.create_table(force=True)

Dropped old table: real_estate_listing
Created new LanceDB table: real_estate_listing


In [11]:
embed_fn = get_embedder()

In [12]:
# Ingest (deduplicated)
manager.ingest_listings(JSON_PATH, embed_fn)

Added 10 new listings


In [13]:
# Try to create index (best-effort)
manager.create_vector_index(index_type="hnsw", metric="cosine")

Could not create index (maybe your LanceDB version lacks this API). LanceTable.create_index() got multiple values for argument 'metric'


In [14]:
# Run a sample RAG query
rag = RAGEngine(manager, embed_fn)
user_query = "Looking for a modern 3-bedroom with good schools, near public transit, under $1.5M"
result = rag.query(user_query, k=5)

LanceDB.search() not vailable of failed: name 'query' is not defined


In [15]:
# Print JSON-safe output
import json
print(json.dumps(result, indent=2))

{
  "answer": "Based on your requirements for a modern 3-bedroom home with good schools, near public transit, and under $1.5M, here are the top 3 recommended listings:\n\n1. Listing ID b90e501a0057fd472627d0a0631e55c4 (score=0.4015):\n   Neighborhood: Oak Hill\n   Price: $780,000\n   Bedrooms: 4, Bathrooms: 3\n   Size: 2,700 sqft\n\n   This listing in Oak Hill offers a stunning 4-bedroom, 3-bathroom modern home with high-end finishes. The neighborhood is known for its excellent schools and family-friendly environment, meeting your criteria for good schools. Additionally, Oak Hill provides easy access to shopping and dining, making it convenient for daily needs. While this property has an extra bedroom, the additional space could be utilized as a home office or guest room.\n\n2. Listing ID 026664fddbf4585689d527cd7ccedcc7 (score=0.3949):\n   Neighborhood: Sunnyvale\n   Price: $450,000\n   Bedrooms: 2, Bathrooms: 1\n   Size: 1,100 sqft\n\n   This charming 2-bedroom, 1-bathroom bungalow i